# Equal Weight S&P 500 Screener

The S&P 500 index is the most popular stock index market in the world. The S&P 500 is market capitalization-weighted.
This poject focuses on building an alternative version of the S&P 500 index fund that assigns the same way to each company.

The goal is to create a Python script that will accept the value of the portfolio and return the required amount of shares each S&P 500 constituent should be purchased to get an equal-weight version of the index fund. 

The larger companies will have lesser weight than the traditional version and the smaller companies will have larger weight in than the traditional version.

In [4]:
import numpy as np
import pandas as pd
import requests         # for http requests
import xlsxwriter 
import math

In [5]:
# importing list of stocks
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Now the IEX Cloud API token is imported in sandbox mode (returns randomized data instead of real time data)

In [2]:
from secrets import IEX_CLOUD_API_TOKEN
#IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

Making the first API call to IEX cloud: 
https://iexcloud.io/docs/api/

The following imformation is retrieved from the API:
 - market cap
 - stock price

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 84823499, 'calculationPrice': 'close', 'change': 2.3, 'changePercent': 0.01684, 'close': 145.66, 'closeSource': 'liofcfia', 'closeTime': 1704563885578, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 143.8, 'delayedPriceTime': 1726056364236, 'extendedChange': 0.09, 'extendedChangePercent': 0.00068, 'extendedPrice': 142.59, 'extendedPriceTime': 1728067473030, 'high': 145.66, 'highSource': 'de ce 1ltderm5ieyinau p', 'highTime': 1682159927371, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 139.84, 'iexCloseTime': 1731037246678, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 139.52, 'iexOpenTime': 1683683868434, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1685485697840, 'latestPrice': 142.48, 'latestSource': 'Close', 'latestTime': 'July 1, 2022', 'latestUpdate': 1718156079597, 'latestVolume': 71840674, 'low': 137.98, 'lowSource': 'ca1iimy der  5nldeeeput

Parsing the API call to get the relevant information

In [8]:
price = data['latestPrice']
market_cap = data['marketCap']

2.282694958976


Adding stocks (price and market capitalization) to a pandas DataFrame. Batch API calls will as otherwise HTTP requests are a very slow component of a script.

In [15]:
my_coloumns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_coloumns)

def chunks(lst, n):
    # Yield successive n-sized chunks from lst
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [30]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_coloumns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW': # there are some errors with these stocks
            continue
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_coloumns),
            ignore_index = True
        )

C:\Users\manis\AppData\Local\Temp\ipykernel_30260\577854356.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\manis\AppData\Local\Temp\ipykernel_30260\577854356.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\manis\AppData\Local\Temp\ipykernel_30260\577854356.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\manis\AppData\Local\Temp\ipykernel_30260\577854356.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\manis\AppData\L

In [31]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,121.51,36050689271,N/A
1,AAL,13.22,8556638213,N/A
2,AAP,177.27,10714160662,N/A
3,AAPL,142.68,2289808291792,N/A
4,ABBV,155.1,275577869745,N/A
...,...,...,...,...
496,YUM,121.93,33632018343,N/A
497,ZBH,110.03,22679181424,N/A
498,ZBRA,312.21,15994153709,N/A
499,ZION,51.63,7919176915,N/A


Calculating number of shares to buy

In [34]:
portfolio_size = input('Enter the value of your portfolio')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Please enter a number \n')
    portfolio_size = input('Enter the value of your portfolio')
    val = float(portfolio_size)

10000000.0


In [42]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,121.51,36050689271,164
1,AAL,13.22,8556638213,1509
2,AAP,177.27,10714160662,112
3,AAPL,142.68,2289808291792,139
4,ABBV,155.1,275577869745,128
...,...,...,...,...
496,YUM,121.93,33632018343,163
497,ZBH,110.03,22679181424,181
498,ZBRA,312.21,15994153709,63
499,ZION,51.63,7919176915,386


#### Now formatting the excel output

Firstly, initializing xlsxwriter object

In [51]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

Creating the formats required for .xlsx files

In [52]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

Applying formats to columns in .xlsx

In [53]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
writer.save()